In [ ]:
#Import dependencies 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import requests
import scipy.stats as stats

In [ ]:
#Read the csv files
olympics_df = pd.read_csv('Resources/athlete_events.csv')
regions_df = pd.read_csv('Resources/noc_regions.csv')
host_cities_df = pd.read_csv('Resources/olym.csv')
arrivals_df = pd.read_csv('Resources/INT.ARVL.csv')
departures_df = pd.read_csv('Resources/INT.DPRT.csv')
expenditures_df = pd.read_csv('Resources/INT.XPND.csv')

In [ ]:
#Jumaan start here (Question 1)

In [ ]:
#Jumaan end here

In [ ]:
#Timmy start here (Question 2)

In [ ]:
#Timmy end here

In [ ]:
#Raj start here (Question 3)

In [ ]:
#Raj end here

In [ ]:
#Fatma start here (Question 4)

 Is there a statistically significant difference between medals won prior to hosting olympics and after hosting?

In [ ]:
# quick look on olympics data

olympics_df.head()

In [ ]:
# look at host cities data
host_cities_df.head()

Clean up mispelled words in the dataframe

In [ ]:
# get only the city and country column from the host city data
host_cities_df = host_cities_df[["City", "Country"]]
host_cities_df.head()

# find the unique city and country names in the host city data frame, and use them to correct spelling mistakes
z=host_cities_df["City"].unique()
y=host_cities_df["Country"].unique()
z
y

In [ ]:
#replace the west germany and nazi germany with germany and soviet union with russia in the host_cities file

host_cities_df = host_cities_df.replace({'Germnay' : 'Germany', 'West Germany': 'Germany', 'Nazi Germany': 'Germany' , 'Soviet Union' : 'Russia'})

#replace the misspelled city names with the correct ones based on the host cities file

olympics_c_df= olympics_df.replace({'Athina':'Athens', 'Antwerpen':'Antwerp', 'Sankt Moritz':'St. Moritz', 'Roma':'Rome', 'Moskva':'Moscow', 'Torino':'Turin'})

#merge olympics and host cities data

olympics_country_df = pd.merge(olympics_c_df, host_cities_df, how="left", on=["City"])


# remove duplicates from olympics data
olympics_country_df = olympics_country_df.drop_duplicates()

# look at data quickly
olympics_country_df.head()



In [ ]:
# leave only the columns that are relevant to the analysis
olympics_country_df = olympics_country_df[["Name", "Team", "Year", "Season", "Sport", "Event", "Medal", "Country"]]
olympics_country_df.shape


In [ ]:
# check if any NaN is left in the Country column. Shape should be equal to olympics_country_df.if there is no NaN left nothing 
# will drop and the row numbers should stay same.
x = olympics_country_df.dropna(subset=["Country"])
x.shape


The medal column has NaN for no medal won. Replace all with 0

In [ ]:
# replace all the NaN in the Medal column with 0
values = {"Medal" : 0}
olympics_country_df = olympics_country_df.fillna(value=values)
olympics_country_df = pd.DataFrame(olympics_country_df)
olympics_country_df.head()


Make a dataframe that shows only teams for the years they hosted and remove any rows when no medal was won

In [ ]:
# Make a new df where the team is from the host country

host_team = olympics_country_df.loc[olympics_country_df["Team"] == olympics_country_df["Country"]]
host_team.head()

    

In [ ]:
# remove the columns with no medal win

medal_win = host_team.loc[host_team["Medal"] != 0]

# drop any duplicate row if there is any
medal_win = medal_win.dropna()

# keep only columns of interest
medal_win = medal_win[[ "Team", "Year", "Medal"]]

In [ ]:
# how many medals host countries won, do groupby

medal_win_host = medal_win.groupby(["Team", "Year"]).count()
medal_win_host

# reset index 

host_year_medal_count = medal_win_host.reset_index()
host_year_medal_count.head()


In [ ]:
# from my corrected host cities dataframe find the unique countries
host_countries = host_cities_df["Country"].unique()
host_countries

Make a dataframe that shows all the teams that hosted olympics anytime, remove rows when no medal was won, count the total medal won for that country in that year

In [ ]:
# clean the olympics dataframe only that it has teams that were also hosts at anytime during the olympics

only_teams_that_hosted = olympics_country_df[olympics_country_df["Team"].isin(olympics_country_df["Country"])]
only_teams_that_hosted.head()

In [ ]:
#remove the columns with no medal win

medal_win_by_team = only_teams_that_hosted.loc[only_teams_that_hosted["Medal"] != 0]
medal_win_by_team = medal_win_by_team.dropna()

#this is different from the medal win dataframe in that this one includes all the years the hosting country participated
#keep only relevant columns

medal_win_by_team = medal_win_by_team[["Team", "Year", "Medal"]]

In [ ]:
# group by team and count the medal wins of the team during the entire time (not only when it is host)

year_medal_count = medal_win_by_team.groupby(["Team", "Year"]).count()
year_medal_count.head()

Make bar graphs showing how many medals each contry won. Color the year of hosting red

In [ ]:
# for loop for each team 

team = only_teams_that_hosted["Team"].unique()
team = team.tolist()
team

for team in team:
    
    teams = year_medal_count.loc[team]
    
    bar = host_year_medal_count.loc[host_year_medal_count["Team"] == team ]["Year"].tolist()
    
    colors = ["red" if x in bar else "yellow" for x in teams.index]
    
    teams.plot.bar(y="Medal", title=f"Olympic medal counts for {team} ", color=colors)
    
    plt.show()
    
    

Do everything only for summer Olympics

In [ ]:
# get the medal wins only for summer olympics for host teams


summer_wins = host_team.loc[host_team["Medal"] != 0]
summer_wins = summer_wins.dropna()
summer_wins

summer_wins = summer_wins.loc[summer_wins["Season"] == "Summer"]
summer_wins = summer_wins[[ "Team", "Year", "Medal"]]
summer_wins

In [ ]:
# medal counts for teams during hosting
summer_win_host = summer_wins.groupby(["Team", "Year"]).count()
summer_win_host

host_year_medal_count_summer = summer_win_host.reset_index()
host_year_medal_count_summer.head()

In [ ]:
# divide the only teams that hosted dataframe into summer

only_teams_that_hosted_summer = only_teams_that_hosted.loc[only_teams_that_hosted["Season"] == "Summer"]
only_teams_that_hosted_summer.head()

In [ ]:
#remove the columns with no medal win

medal_win_by_team_summer = only_teams_that_hosted_summer.loc[only_teams_that_hosted_summer["Medal"] != 0]
medal_win_by_team_summer= medal_win_by_team_summer.dropna()
medal_win_by_team_summer = medal_win_by_team_summer[["Team", "Year", "Medal"]]

In [ ]:
# group by team and count the medal wins of the team during the entire time (not only when it is host)

year_medal_count_summer = medal_win_by_team_summer.groupby(["Team", "Year"]).count()
year_medal_count_summer.head()

In [ ]:
# for loop for each team 

team = only_teams_that_hosted_summer["Team"].unique()
team = team.tolist()
team

for team in team:
    
    teams = year_medal_count_summer.loc[team]
    
    bar = host_year_medal_count_summer.loc[host_year_medal_count_summer["Team"] == team ]["Year"].tolist()
    
    colors = ["red" if x in bar else "yellow" for x in teams.index]
    
    teams.plot.bar(y="Medal", title=f"Olympic medal counts for {team} ", color=colors)
    
    plt.show()
    

Calculate Ttest for checking the null hypothesis that states "The mean medal win for a country before and after hosting the Olympics is NOT different"

In [ ]:
# create a list of dataframes that will show the country, year they played and medals they won
# first remove all countries that hosted for more than ones. 

j =year_medal_count_summer.reset_index()
t= host_year_medal_count_summer


t["Team"].value_counts() == 1
r = t.drop_duplicates(subset = "Team", keep=False)
r

# make a list that contains these countries
team_summer = r["Team"].tolist()

In [ ]:
# now make a for loop that will show the country and medal wins the years after they hosted.
team_list_after = []

for team in team_summer:
    
    df = j.loc[(j["Team"] == team) & (j["Year"] > r[r['Team']==team]['Year'].values[0])]
    team_list_after.append(df)
    
    
    
team_list_after[0]

In [ ]:
# now make a for loop that will show the country and medal wins the years before they hosted.


team_list_before = []

for team in team_summer:
    
    df = j.loc[(j["Team"] == team) & (j["Year"] < r[r['Team']==team]['Year'].values[0])]
    team_list_before.append(df)
    
    
    
team_list_before[0]["Medal"]

In [ ]:
# calculate t test for each country data obtained from the for loops above
#make a for loop for each team

teams = []
for i in range(len(team_list_after)):

    test = stats.ttest_ind(team_list_after[i].Medal, team_list_before[i].Medal, equal_var=False)
    teamss = team_list_after[i]["Team"].unique().tolist()
    teams.append(teamss)
    
    print(f"Ttest result for {teams[i]}: {test}")

We were able to reject our null hypothesis for Belgium, Canada, China, Finland, Germany,Japan, Netherlands, South Korea and Spain. 
We failed to reject the null hypothesis for Italy and Mexico.
We did not have enough data to perform the test on Brazil.

In [ ]:
#Fatma end here